In [1]:
import gzip
import pandas as pd
import os
import random
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, classification_report, r2_score
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

In [2]:
def load_initial_data(path):
    df = []
    for line in gzip.open(path, 'rb'):
        df.append(eval(line))
    return df

In [3]:
data = []
for file in os.listdir("amazon"):
    data+=load_initial_data("amazon/"+file)
    



In [4]:
full_data = data

In [5]:
data = full_data

In [6]:
data = data[:int(len(data)*0.05)]
random.shuffle(data)
dataframe = pd.DataFrame.from_dict(data)

In [7]:
dataframe.head()

,answer,answerTime,answerType,asin,question,questionType,unixTime
0,I don't think you'd want to use it to cut stea...,"Jun 6, 2014",?,B00005AKZK,Does it come pre sharpened? It seems like it m...,yes/no,1.402038e+09
1,i don't know the BTU out put but when i hook i...,"May 24, 2014",NaN,B005CXE650,How many BTU is it? I have to size the regulat...,open-ended,1.400915e+09
2,"No, they sre water resistant.","Oct 7, 2013",N,B008BDEQEY,Is this water proof?,yes/no,1.381129e+09
3,I don't see why not. It is pretty sturdy and d...,"Mar 27, 2015",?,B000N4PH6K,can i use this for a diaper bag?,yes/no,1.427440e+09
4,I am using the solar panels on two different p...,"Mar 23, 2015",NaN,B00BFGG6CY,my pump is 1/2 HP. Can this solar heater handl...,open-ended,1.427094e+09


In [8]:
X_train, Y_train = dataframe["question"], dataframe["questionType"]


In [9]:
index = np.random.rand(len(dataframe)) < 0.8
index

array([False,  True, False, ...,  True,  True,  True])

In [10]:
TRAIN = dataframe[index]
TEST = dataframe[~index]

In [11]:
len(TRAIN)

55950

In [12]:
len(TEST)

13894

In [13]:
X_train, Y_train = TRAIN["question"], TRAIN["questionType"]
X_test, Y_test = TEST["question"], TEST["questionType"]

Y_train  = Y_train.apply(lambda y: 1 if y == "yes/no" else 0).as_matrix()
Y_test  = Y_test.apply(lambda y: 1 if y == "yes/no" else 0).as_matrix()

/home/ostapkharysh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/home/ostapkharysh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [14]:
#Parameters
jobs = [1, 3, 5]


In [16]:
#lin_model = linear_model.LinearRegression()
#mdl_lin = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3))), ('lr', linear_model.LinearRegression())])
lin_df = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3))), ('lr', linear_model.LinearRegression())])
#mdl_log = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3))), ('lr', linear_model.LogisticRegression())])
grid = GridSearchCV(cv = 3, estimator = lin_df, param_grid = dict(lr__n_jobs =jobs,))
lin_df.get_params().keys()

dict_keys(['lr__fit_intercept', 'lr__n_jobs', 'tfidf__tokenizer', 'tfidf__strip_accents', 'tfidf__lowercase', 'tfidf__stop_words', 'tfidf__smooth_idf', 'tfidf__input', 'tfidf__use_idf', 'tfidf', 'lr', 'lr__normalize', 'tfidf__preprocessor', 'tfidf__token_pattern', 'tfidf__max_features', 'tfidf__max_df', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__norm', 'tfidf__min_df', 'tfidf__analyzer', 'tfidf__sublinear_tf', 'steps', 'tfidf__binary', 'memory', 'tfidf__encoding', 'tfidf__ngram_range', 'tfidf__vocabulary', 'lr__copy_X'])

In [ ]:
grid.fit(X_train, Y_train)
#mdl_lin.get_params().keys()

In [ ]:
pred_lin = grid.predict(X_test)
#pred_log = mdl_log.predict(X_test)

In [ ]:
print("LINEAR REGRESSION")
print("Accuracy: %.5f" % r2_score(Y_test, pred_lin))
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, pred_lin))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_test, pred_lin))
print()
"""

print("LOGISTIC REGRESSION")
print("Accuracy: %.5f" % r2_score(Y_test, pred_log))
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, pred_log))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_test, pred_log))
"""